In [1]:
import netCDF4
import numpy as np
import time
import datetime

In [2]:
# ncd = netCDF4.Dataset("http://thredds.aoos.org/thredds/dodsC/PMEL_CORE.nc")
# outname = 'PMEL_CORE_1994-2004.nc'
# title='CLIVAR CORE climate hindcast for temperature and salinity, 1994-2004'
# description = 'A subset of climate hindcasts from the CLIVAR CORE Climate Model'
# history = 'Source files created by PMEL. These files were subset by time, depth, and variable by William Koeppen (Axiom Data Science) will@axiomdatascience.com (907)231-0925'
# reference = 'http://portal.aoos.org/#module-metadata/5626a0b6-7d79-11e3-ac17-00219bfe5678/0756e6c2-a8e2-40af-aa3d-22051ed68067'
# # Subset by time
# start_day = datetime.datetime(1994,1,1,0,0)
# end_day = datetime.datetime(2004,12,31,0,0)

# ncd = netCDF4.Dataset("http://thredds.aoos.org/thredds/dodsC/PMEL_CCCMA1.nc")
# outname = 'PMEL_CCCma_2029-2039.nc'
# title='PMEL CCCma climate forecast for temperature and salinity, 2039-2049'
# description = 'A subset of climate forecasts from the PMEL CCCma Climate Model'
# history = 'Source files created by PMEL. These files were subset by time, depth, and variable by William Koeppen (Axiom Data Science) will@axiomdatascience.com (907)231-0925'
# reference = 'http://portal.aoos.org/#module-metadata/4f706756-7d57-11e3-bce5-00219bfe5678/ffa1bcc1-288d-4f8e-912e-500a618b241a'
# # Subset by time
# start_day = datetime.datetime(2029,1,1,0,0)
# end_day = datetime.datetime(2039,12,31,0,0)

# ncd = netCDF4.Dataset("http://thredds.aoos.org/thredds/dodsC/PMEL_MIROC.nc")
# outname = 'PMEL_MIROC_2029-2039.nc'
# title='PMEL MIROC climate forecast for temperature and salinity, 2039-2049'
# description = 'A subset of climate forecasts from the PMEL MIROC Climate Model'
# history = 'Source files created by PMEL. These files were subset by time, depth, and variable by William Koeppen (Axiom Data Science) will@axiomdatascience.com (907)231-0925'
# reference = 'http://portal.aoos.org/#module-metadata/68ea728a-7d7a-11e3-823b-00219bfe5678/bb0d0b5e-878f-4ebb-8985-0d0e6aefe71f'
# # Subset by time
# start_day = datetime.datetime(2029,1,1,0,0)
# end_day = datetime.datetime(2039,12,31,0,0)

ncd = netCDF4.Dataset("http://thredds.aoos.org/thredds/dodsC/PMEL_ECHOG.nc")
outname = 'PMEL_ECHOG_2029-2039.nc'
title='PMEL ECHO-G climate forecast for temperature and salinity, 2039-2049'
description = 'A subset of climate forecasts from the PMEL ECHO-G Climate Model'
history = 'Source files created by PMEL. These files were subset by time, depth, and variable by William Koeppen (Axiom Data Science) will@axiomdatascience.com (907)231-0925'
reference = 'http://portal.aoos.org/#module-metadata/18ffa59c-7d7a-11e3-82a4-00219bfe5678/f2e5592b-28d2-483d-8ef8-52aa18f6e3dc'
# Subset by time
start_day = datetime.datetime(2029,1,1,0,0)
end_day = datetime.datetime(2039,12,31,0,0)

In [3]:
# Get time
time_var = ncd.variables['TIME']
times=netCDF4.num2date(np.array(time_var), time_var.units)

time_index=(times >= start_day) & (times <= end_day)
times_subset=times[time_index]

# the times_subset above should work, but it doesn't allow me to move to a range, so I'll use this instead:
start_tindex = np.searchsorted(times, start_day, side='left')
end_tindex = np.searchsorted(times, end_day, side='right')
ntimes=end_tindex-start_tindex
print('Number of times: ', ntimes)

Number of times:  557


In [4]:
# Get depths
depth_var = ncd.variables['zsalt']
depths=np.array(depth_var)

# Subset by vertical extent
start_depth = 0
end_depth = 10

depth_index=(depths >= start_depth) & (depths <= end_depth)
depth_subset=depths[depth_index]

# the dpeth_subset above should work, but it doesn't allow me to move to a range, so I'll use this instead:
start_dindex = np.searchsorted(depths, start_depth, side='left')
end_dindex = np.searchsorted(depths, end_depth, side='right')
ndepths=end_dindex-start_dindex
print('Number of depths: ', ndepths)

Number of depths:  3


In [5]:
# Pull the temperature variable. There's a size restriction on the request,
# so I have to do it in two gos and put it back together.
temp_var = ncd.variables['sea_water_temperature']

In [6]:
# If I index by [0] rather than 0, it leaves the depth axis intact instead of removing it.
temps1d = np.array(temp_var[start_tindex:end_tindex, [0], :,:])
temps2d = np.array(temp_var[start_tindex:end_tindex, [1], :,:])
temps3d = np.array(temp_var[start_tindex:end_tindex, [2], :,:])
temps=np.concatenate((temps1d, temps2d, temps3d), axis=1)

In [7]:
temps.shape

(557, 3, 161, 251)

In [8]:
# Pull the salinity variable. There's a size restriction on the request,
# so I have to do it in two gos and put it back together.
sal_var = ncd.variables['salinity']
sals1d = np.array(sal_var[start_tindex:end_tindex, [0], :,:])
sals2d = np.array(sal_var[start_tindex:end_tindex, [1], :,:])
sals3d = np.array(sal_var[start_tindex:end_tindex, [2], :,:])
sals=np.concatenate((sals1d, sals2d, sals3d), axis=1)

In [9]:
sals.shape

(557, 3, 161, 251)

In [10]:
# Write out to netCDF
# Add dimensions
ncfile = netCDF4.Dataset(outname, 'w')
ncfile.createDimension('TIME', )
ncfile.createDimension('DEPTH', len(depth_subset))
ncfile.createDimension('LATITUDE', ncd.dimensions['LATITUDE'].size)
ncfile.createDimension('LONGITUDE', ncd.dimensions['LONGITUDE'].size)

# Add subsetted time variable
## Write it out
nc_times=ncfile.createVariable('TIME', float, dimensions=('TIME'))
nc_times.standard_name = 'time'
nc_times.units = time_var.units
nc_times.time_origin = time_var.time_origin
nc_times[:] = (np.array(time_var))[time_index]

# Add subsetted depth variable
## Write it out
nc_depths=ncfile.createVariable('DEPTH', float, dimensions=('DEPTH'))
nc_depths.standard_name = 'depth'
nc_depths.units = depth_var.units
nc_depths.positive = 'down'
nc_depths[:] = depth_subset

# Add latitude variable
## Read it in
lat = np.array(ncd.variables['LATITUDE'])
## Write it out
nc_lats=ncfile.createVariable('LATITUDE', float, dimensions=('LATITUDE'))
nc_lats.standard_name = 'latitude'
nc_lats.units = 'degrees_north'
nc_lats.point_spacing = 'even'
nc_lats.axis = 'Y'
nc_lats[:]=lat

# Add longitude variable
## Read it in
lon = np.array(ncd.variables['LONGITUDE'])
## Write it out
nc_lons=ncfile.createVariable('LONGITUDE', float, dimensions=('LONGITUDE'))
nc_lons.standard_name = 'longitude'
nc_lons.units = 'degrees_east'
nc_lons.modulo = 360.
nc_lons.axis = 'X'
nc_lons[:] = lon

# Add temperature
# Write
nc_temps = ncfile.createVariable('sea_water_temperature', float, dimensions=('TIME', 'DEPTH', 'LATITUDE', 'LONGITUDE'))
nc_temps.long_name = 'time-averaged potential temperature'
nc_temps.units = temp_var.units
nc_temps._fillValue = temp_var._FillValue
nc_temps[:,:,:,:] = temps[:,:,:,:]

# Add salinity
# Write
nc_sals = ncfile.createVariable('salinity', float, dimensions=('TIME', 'DEPTH', 'LATITUDE', 'LONGITUDE'))
nc_sals.long_name = 'time-averaged salinity'
nc_sals.units = sal_var.units
nc_sals._fillValue = sal_var._FillValue
nc_sals[:,:,:,:] = sals[:,:,:,:]

# Add Attributes
ncfile.title = title
ncfile.description = description
ncfile.history = history
ncfile.reference = reference

ncfile.close()